# Mapping

## Aligner les reads

Objectif : Trouver la région du génome qui a produit les read
- Trouver dans le génome le mot correspondant au read

![](images/ebaii_mapping.png)

## L’approche de bowtie: seed and extend

Une extrémité du read est interrogée (la graine) 

![](images/ebaii_mapping_readSeed.png)

On cherche ses régions correspondantes sur le génome (à l’aide d’un index créé initialement) avec ou sans mismatch.

On teste si le reste du read s’aligne avec la séquence

![](images/ebaii_mapping_positions.png)

## Aligner les reads

Pour l’alignement nous utiliserons Bowtie 2

Bowtie 2 nécessite de préparer un index
- Cet index permet une recherche optimisée de la position d’un mot w dans le génome
- Des index pour les génomes utilisés classiquement sont disponibles sur le site de bowtie 2 et aussi dans les banques du cluster (ex. `/shared/bank/homo_sapiens/hg38/`)
- Ici nous voulons restreindre le génome au chromosome 21, nous devons donc construire cet index

In [1]:
# Créez un répertoire pour y stocker l’index dans chip-seq/
cd /shared/projects/form_2022_32/coursLinux/chip-seq/   
mkdir index
cd index

## Création de l'index

```{warning}
Ne faire qu’une seule fois par génome d’intérêt et version majeure !
```

1. Allez sur le site de l’UCSC à l’adresse suivante
- https://genome.ucsc.edu/
2. Cliquez sur Downloads > Genome Data > human > hg38 > Data set by chromosome. 
3. Recherchez le fichier chr21.fa.gz
4. Cliquez bouton droit “Copy link address”

In [2]:
# Téléchargez l’index avec wget
wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr21.fa.gz 

--2022-11-10 18:47:13--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr21.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12709705 (12M) [application/x-gzip]
Saving to: ‘chr21.fa.gz’

100%[======================================>] 12,709,705  7.39MB/s   in 1.6s   

2022-11-10 18:47:16 (7.39 MB/s) - ‘chr21.fa.gz’ saved [12709705/12709705]



In [3]:
# décompression
gunzip chr21.fa.gz

In [4]:
module load bowtie2/2.3.4.3 samtools/1.9		# ici on charge 2 outils à la fois
# Construction de l’index
bowtie2-build chr21.fa chr21_hg38

Settings:
  Output files: "chr21_hg38.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  chr21.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 10022154
Using parameters --bmax 7516616 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 7516616 --dcv 1024
Constructing suffix-array eleme

## Alignement

On crée un répertoire de travail et on se positionne dans celui-ci
On lancera l’alignement dans depuis le dossier ‘bam’.

In [5]:
# Create a directory 
mkdir -p ../bam 
# Change directory 
cd ../bam 

## Alignement (do not run!)

```{note}
Pour des raison pédagogique, nous allons diviser la commande en plusieurs blocs.
```

### Partie 1

1. L’alignement est réalisé avec bowtie2, qui produit un flux de texte au format sam (texte), volumineux.

```bash
# Perform alignment
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log 
```

### Partie 2

1. L’alignement est réalisé avec bowtie2, qui produit un flux de texte au format sam (texte), volumineux.
2. Ce flux de texte peut être redirigé (|) vers 'samtools view -hbS' (-h: header, -b output is BAM, -S: input is SAM) pour produire une version compressée (format bam).

```bash
# -bS (sortie en bam, entrée en sam) 
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log | samtools view -hbS
```

### Partie 3

1. L’alignement est réalisé avec bowtie2, qui produit un flux de texte au format sam (texte), volumineux.
2. Ce flux de texte peut être redirigé (|) vers 'samtools view -hbS' (-h: header, -b output is BAM, -S: input is SAM) pour produire une version compressée (format bam).
3. On sélectionne le sous-ensemble des reads pour lequel la mapping quality (-q: quality) est au moins égale à 30. 

```bash 
# -q 30 (quality 30)
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log | samtools view -hbS -q 30
```

### Partie 4 

1. L’alignement est réalisé avec bowtie2, qui produit un flux de texte au format sam (texte), volumineux.
2. Ce flux de texte peut être redirigé (|) vers 'samtools view -hbS' (-h: header, -b output is BAM, -S: input is SAM) pour produire une version compressée (format bam).
3. On sélectionne le sous-ensemble des reads pour lequel la mapping quality (-q: quality) est au moins égale à 30.
4. Le flux de texte est redirigé (|) vers `samtools sort` (trie par coordonnées génomiques).


```bash 
# Trie l’alignement
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log | samtools view -hbS -q 30 | samtools sort
```

### Partie 5 

1. L’alignement est réalisé avec bowtie2, qui produit un flux de texte au format sam (texte), volumineux.
2. Ce flux de texte peut être redirigé (|) vers 'samtools view -hbS' (-h: header, -b output is BAM, -S: input is SAM) pour produire une version compressée (format bam).
3. On sélectionne le sous-ensemble des reads pour lequel la mapping quality (-q: quality) est au moins égale à 30.
4. Le flux de texte est redirigé (|) vers `samtools sort` (trie par coordonnées génomiques).
5. Le flux de texte est redirigé dans un fichier (‘>’)


```bash 
# ‘>’ est un opérateur de redirection 
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log | samtools view -hbS -q 30 | samtools sort \
  > siNT_ER_E2_r3_chr21_trim.bam
```

### Partie 6 

1. L’alignement est réalisé avec bowtie2, qui produit un flux de texte au format sam (texte), volumineux.
2. Ce flux de texte peut être redirigé (|) vers 'samtools view -hbS' (-h: header, -b output is BAM, -S: input is SAM) pour produire une version compressée (format bam).
3. On sélectionne le sous-ensemble des reads pour lequel la mapping quality (-q: quality) est au moins égale à 30.
4. Le flux de texte est redirigé (|) vers `samtools sort` (trie par coordonnées génomiques).
5. Le flux de texte est redirigé dans un fichier (‘>’)
6. Le fichier est indexé pour optimiser la recherche de position dans le BAM (création d’un fichier *.bai).

```bash 
# Indexation de l’alignement
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log | samtools view -hbS -q 30 | samtools sort \
  > siNT_ER_E2_r3_chr21_trim.bam
```


### Commande finale

In [6]:
bowtie2 -p 4 -x ../index/chr21_hg38 -U ../trimmed/siNT_ER_E2_r3_chr21_trim.fastq \
  2> siNT_ER_E2_r3_chr21_trim_bowtie2.log | samtools view -hbS -q 30 | samtools sort \
  > siNT_ER_E2_r3_chr21_trim.bam

In [7]:
samtools index siNT_ER_E2_r3_chr21_trim.bam

In [8]:
ls

core.47529		      siNT_ER_E2_r3_chr21_trim.bam.bai
siNT_ER_E2_r3_chr21_trim.bam  siNT_ER_E2_r3_chr21_trim_bowtie2.log


## Fichier bam

- SAM: ‘Sequence Alignment/MAP’
- BAM: binary/compressed version of SAM
- Stocke les informations liées à l’alignement
    - Coordonnées du read aligné
    - Mapping quality
    - CIGAR String
    - Bitwise FLAG
        - read paired, read mapped in proper pair, read unmapped, ...
    - ...
## Visualiser le contenu du fichier bam

Le fichier bam est compressé. On peut voir son contenu avec la commande samtools.

```bash 
# Visualiser le contenu du fichier bam
# On utilise l’argument -h pour visualiser aussi le ‘header’.
# On renvoie le flux de texte dans less. 
# On ajoute le paramètre -S pour tronquer les lignes qui excèdent
# la largeur de l’écran 
samtools view -h siNT_ER_E2_r3_chr21_trim.bam | less -S
```

## Bitwise flag
De nombreuses informations sont stockées dans la colonne 2 du fichier SAM/BAM
- read pairs
- reads mapped in proper pairs
- reads unmapped
- mates unmapped
- reads reverse strand
- mates reverse strand
- first in pair
- second in pair
- not primary alignment
- ...

Détails:
- 00000000001 → 2^0  = 1 (read paired)
- 00000000010 → 2^1  = 2 (read mapped in proper pair)
- 00000000100 → 2^2  = 4 (read unmapped)
- 00000001000 → 2^3  = 8 (mate unmapped) …
- 00000010000 → 2^4  = 16 (read reverse strand)
- 00000001001 →  2^0+ 2^3 = 9 → (read paired, mate unmapped)
- 00000001101 → 2^0+2^2+2^3 =13 ...
- ...


```{seealso}
Voir (Explained SAM flags)[https://broadinstitute.github.io/picard/explain-flags.html]
```

## The extended CIGAR string

Quelques exemples de drapeaux (flag)
- M : match ou mismatch...
- I : Insertion par rapport à la référence
- D : Délétion par rapport à la référence
- N : Espace dans l’alignement (Gap)

```{seealso}
Voir (SAM)[http://samtools.sourceforge.net/SAM1.pdf]
```

```
ATTCAGATGCAGTA
ATTCA--TGCAGTA
```

```
5M2D7M
```

## Pourquoi filtrer sur la qualité ?

Sommes-nous plus confiants 
- dans l’alignement du read 1 ?
- dans l’alignement read 2 ?

![](images/filter_quality.png)

Sommes-nous plus confiants dans l’alignement 1 ?
- Si la moyenne de qualité des nucléotides séquencés dans le read est 40 dans l’alignement 1’ ?
- Si la moyenne de qualité des nucléotides séquencés dans le read est 10 ? 

![](images/filter_quality2.png)

## Filtering for Mapping Quality (MAPQ)

Mapping quality is a score that integrates both the quality of the read itself and the number of positions it maps

Mapping quality score is computed from the probability that alignment is wrong:
- takes mappability and sequence quality into account
- -10.log10(Prob(alignment is wrong))
    - p=0.01 -> MAPQ: 20
    - p=0.001 -> MAPQ: 30
    - p=0.0001 -> MAPQ: 40
    - ...
